# Machine-learning Notebook
- preceeded by feature creation notebooks

In [3]:
import pandas as pd
import numpy as np
import itertools
import matplotlib.pyplot as plt
%matplotlib inline
import welly
from welly import Well
import lasio
import glob
from sklearn import neighbors
import pickle
import math
import dask
import dask.dataframe as dd
from dask.distributed import Client
# import pdvega
# import vega
import random
from xgboost.sklearn import XGBClassifier
from sklearn.metrics import mean_squared_error


In [4]:
print(welly.__version__)
print(dask.__version__)
print(pd.__version__)

0.3.5
0.18.1
0.23.1


In [5]:
%%timeit
import os
env = %env

79.2 µs ± 1.23 µs per loop (mean ± std. dev. of 7 runs, 10000 loops each)


In [6]:
#### Had to change display options to get this to print in full!
#pd.set_option('display.height', 1000)
pd.set_option('display.max_rows', 500)
pd.set_option('display.max_columns', 500)
pd.set_option('display.width', 1000)
pd.options.display.max_colwidth = 100000

In [7]:
knn_dir = "../WellsKNN/"
load_dir = "../loadLAS"
features_dir = "../createFeatures/"

## If you open this notebook fresh and jump to a point below where a pick file is read in, you still need to load everything above! 

-------------

------------

## Reading in the last hdf5 file

In [8]:
h5_to_load = 'df_all_wells_wKNN_DEPTHtoDEPT_KNN1PredTopMcM_20180724.h5'
h5_key = 'df'
df_all_Col_preSplit = pd.read_hdf(features_dir+h5_to_load, h5_key)

In [9]:
df_all_Col_preSplit.head()

,CALI,COND,DELT,DENS,DEPT,DEPTH,DPHI,DPHI:1,DPHI:2,DT,GR,GR:1,GR:2,IL,ILD,ILD:1,ILD:2,ILM,LITH,LLD,LLS,NPHI,PHID,PHIN,RESD,RHOB,RT,SFL,SFLU,SN,SNP,SP,UWI,SitID,McMurray_Base_HorID,McMurray_Top_HorID,McMurray_Base_DEPTH,McMurray_Top_DEPTH,McMurray_Base_Qual,McMurray_Top_Qual,lat,lng,NN1_McMurray_Top_DEPTH,NN1_McMurray_Base_DEPTH,NN1_thickness,MM_Top_Depth_predBy_NN1thick,HorID,Pick,Quality,HorID_paleoz,Pick_paleoz,Quality_paleoz,diff_TMcM_Pick_v_DEPT,diff_TPal_Pick_v_DEPT,cat_isTopMcMrNearby_known,cat_isTopPalNearby_known,DistFrom_NN1_TopDepth_Abs,NewWell,LastBitWell,TopWellDept,BotWellDept,FromTopWell,FromBotWell,WellThickness,closerToBotOrTop,closTopBotDist,rowsToEdge,GR_min_5winSize_dirAroundMin,GR_min_5winSize_dirAboveMin,GR_min_5winSize_dirAroundMax,GR_min_5winSize_dirAboveMax,GR_min_5winSize_dirAroundMean,GR_min_5winSize_dirAboveMean,GR_min_5winSize_dirAbovenLarge,GR_min_5winSize_dirAroundnLarge,GR_min_7winSize_dirAroundMin,GR_min_7winSize_dirAboveMin,GR_min_7winSize_dirAroundMax,GR_min_7winSize_dirAboveMax,GR_min_7winSize_dirAroundMean,GR_min_7winSize_dirAboveMean,GR_min_7winSize_dirAbovenLarge,GR_min_7winSize_dirAroundnLarge,GR_min_11winSize_dirAroundMin,GR_min_11winSize_dirAboveMin,GR_min_11winSize_dirAroundMax,GR_min_11winSize_dirAboveMax,GR_min_11winSize_dirAroundMean,GR_min_11winSize_dirAboveMean,GR_min_11winSize_dirAbovenLarge,GR_min_11winSize_dirAroundnLarge,GR_min_21winSize_dirAroundMin,GR_min_21winSize_dirAboveMin,GR_min_21winSize_dirAroundMax,GR_min_21winSize_dirAboveMax,GR_min_21winSize_dirAroundMean,GR_min_21winSize_dirAboveMean,GR_min_21winSize_dirAbovenLarge,GR_min_21winSize_dirAroundnLarge,ILD_min_5winSize_dirAroundMin,ILD_min_5winSize_dirAboveMin,ILD_min_5winSize_dirAroundMax,ILD_min_5winSize_dirAboveMax,ILD_min_5winSize_dirAroundMean,ILD_min_5winSize_dirAboveMean,ILD_min_5winSize_dirAbovenLarge,ILD_min_5winSize_dirAroundnLarge,ILD_min_7winSize_dirAroundMin,ILD_min_7winSize_dirAboveMin,ILD_min_7winSize_dirAroundMax,ILD_min_7winSize_dirAboveMax,ILD_min_7winSize_dirAroundMean,ILD_min_7winSize_dirAboveMean,ILD_min_7winSize_dirAbovenLarge,ILD_min_7winSize_dirAroundnLarge,ILD_min_11winSize_dirAroundMin,ILD_min_11winSize_dirAboveMin,ILD_min_11winSize_dirAroundMax,ILD_min_11winSize_dirAboveMax,ILD_min_11winSize_dirAroundMean,ILD_min_11winSize_dirAboveMean,ILD_min_11winSize_dirAbovenLarge,ILD_min_11winSize_dirAroundnLarge,ILD_min_21winSize_dirAroundMin,ILD_min_21winSize_dirAboveMin,ILD_min_21winSize_dirAroundMax,ILD_min_21winSize_dirAboveMax,ILD_min_21winSize_dirAroundMean,ILD_min_21winSize_dirAboveMean,ILD_min_21winSize_dirAbovenLarge,ILD_min_21winSize_dirAroundnLarge
0,167.003,NaN,NaN,NaN,149.602,NaN,0.227,NaN,NaN,NaN,102.473,NaN,NaN,NaN,0.000,NaN,NaN,NaN,NaN,NaN,NaN,0.460,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,00/10-32-080-20W4/0,112385,14000,13000,384.66,377.95,1,3,55.978836,-113.095365,389.0,414.0,25.0,359.66,13000,377.95,3,14000,384.66,1,228.348,235.058,0,0,210.058,True,False,149.602,396.102,0.00,246.50,246.5,FromTopWell,0.00,0,102.473,102.473,102.473,102.473,102.473,102.473,102.473,102.473,102.473,102.473,102.473,102.473,102.473,102.473,102.473,102.473,102.473,102.473,102.473,102.473,102.473,102.473,102.473,102.473,102.473,102.473,102.473,102.473,102.473,102.473,102.473,102.473,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000
1,199.159,NaN,NaN,NaN,149.852,NaN,0.263,NaN,NaN,NaN,122.589,NaN,NaN,NaN,4.202,NaN,NaN,NaN,NaN,NaN,NaN,0.550,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,00/10-32-080-20W4/0,112385,14000,13000,384.66,377.95,1,3,55.978836,-113.095365,389.0,414.0,25.0,359.66,13000,377.95,3,14000,384.66,1,228.098,234.808,0,0,209.808,False,False,149.602,396.102,0.25,246.25,246.5,FromTopWell,0.25,1,122.589,122.589,122.589,122.589,122.589,122.589,122.589,122.589,122.589,122.589,122.589,122.589,122.589,122.589,122.589,122.589,122.589,122.589,122.589,122.589,122.589,122.589

In [10]:
# df_all_wells_wKNN_DEPTHtoDEPT_KNN1PredTopMcM_20180724_initial.head()

-------------

## Because we have a lot more data points away from the pick (one class) than exactly at the pick or close to the pick (another class), we run the risk of being class heavy. We'll attemp to deal with this problem by only using some of the data not near or at a pick

Get all the UWIs

In [11]:
UWIs = list(df_all_Col_preSplit['UWI'].unique())

Find the number of wells if you want 80%

In [12]:
numberOfTrainingWells = math.floor(len(UWIs)*0.8)
numberOfTrainingWells

1525

Randomly select that number of UWIs for training and the ones left for test

In [13]:
UWIs_training = random.sample(UWIs, numberOfTrainingWells)

In [14]:
UWIs_test = [x for x in UWIs if x not in UWIs_training]

In [15]:
print("train",len(UWIs_training))
print("test",len(UWIs_test))

train 1525
test 382


In [16]:
df_all_Col_preSplit_wTrainTest = df_all_Col_preSplit.copy()

In [17]:
df_all_Col_preSplit_wTrainTest['trainOrTest'] = np.where(df_all_Col_preSplit_wTrainTest['UWI'].isin(UWIs_training), 'train', 'test')

In [18]:
df_all_Col_preSplit_wTrainTest.tail()

,CALI,COND,DELT,DENS,DEPT,DEPTH,DPHI,DPHI:1,DPHI:2,DT,GR,GR:1,GR:2,IL,ILD,ILD:1,ILD:2,ILM,LITH,LLD,LLS,NPHI,PHID,PHIN,RESD,RHOB,RT,SFL,SFLU,SN,SNP,SP,UWI,SitID,McMurray_Base_HorID,McMurray_Top_HorID,McMurray_Base_DEPTH,McMurray_Top_DEPTH,McMurray_Base_Qual,McMurray_Top_Qual,lat,lng,NN1_McMurray_Top_DEPTH,NN1_McMurray_Base_DEPTH,NN1_thickness,MM_Top_Depth_predBy_NN1thick,HorID,Pick,Quality,HorID_paleoz,Pick_paleoz,Quality_paleoz,diff_TMcM_Pick_v_DEPT,diff_TPal_Pick_v_DEPT,cat_isTopMcMrNearby_known,cat_isTopPalNearby_known,DistFrom_NN1_TopDepth_Abs,NewWell,LastBitWell,TopWellDept,BotWellDept,FromTopWell,FromBotWell,WellThickness,closerToBotOrTop,closTopBotDist,rowsToEdge,GR_min_5winSize_dirAroundMin,GR_min_5winSize_dirAboveMin,GR_min_5winSize_dirAroundMax,GR_min_5winSize_dirAboveMax,GR_min_5winSize_dirAroundMean,GR_min_5winSize_dirAboveMean,GR_min_5winSize_dirAbovenLarge,GR_min_5winSize_dirAroundnLarge,GR_min_7winSize_dirAroundMin,GR_min_7winSize_dirAboveMin,GR_min_7winSize_dirAroundMax,GR_min_7winSize_dirAboveMax,GR_min_7winSize_dirAroundMean,GR_min_7winSize_dirAboveMean,GR_min_7winSize_dirAbovenLarge,GR_min_7winSize_dirAroundnLarge,GR_min_11winSize_dirAroundMin,GR_min_11winSize_dirAboveMin,GR_min_11winSize_dirAroundMax,GR_min_11winSize_dirAboveMax,GR_min_11winSize_dirAroundMean,GR_min_11winSize_dirAboveMean,GR_min_11winSize_dirAbovenLarge,GR_min_11winSize_dirAroundnLarge,GR_min_21winSize_dirAroundMin,GR_min_21winSize_dirAboveMin,GR_min_21winSize_dirAroundMax,GR_min_21winSize_dirAboveMax,GR_min_21winSize_dirAroundMean,GR_min_21winSize_dirAboveMean,GR_min_21winSize_dirAbovenLarge,GR_min_21winSize_dirAroundnLarge,ILD_min_5winSize_dirAroundMin,ILD_min_5winSize_dirAboveMin,ILD_min_5winSize_dirAroundMax,ILD_min_5winSize_dirAboveMax,ILD_min_5winSize_dirAroundMean,ILD_min_5winSize_dirAboveMean,ILD_min_5winSize_dirAbovenLarge,ILD_min_5winSize_dirAroundnLarge,ILD_min_7winSize_dirAroundMin,ILD_min_7winSize_dirAboveMin,ILD_min_7winSize_dirAroundMax,ILD_min_7winSize_dirAboveMax,ILD_min_7winSize_dirAroundMean,ILD_min_7winSize_dirAboveMean,ILD_min_7winSize_dirAbovenLarge,ILD_min_7winSize_dirAroundnLarge,ILD_min_11winSize_dirAroundMin,ILD_min_11winSize_dirAboveMin,ILD_min_11winSize_dirAroundMax,ILD_min_11winSize_dirAboveMax,ILD_min_11winSize_dirAroundMean,ILD_min_11winSize_dirAboveMean,ILD_min_11winSize_dirAbovenLarge,ILD_min_11winSize_dirAroundnLarge,ILD_min_21winSize_dirAroundMin,ILD_min_21winSize_dirAboveMin,ILD_min_21winSize_dirAroundMax,ILD_min_21winSize_dirAboveMax,ILD_min_21winSize_dirAroundMean,ILD_min_21winSize_dirAboveMean,ILD_min_21winSize_dirAbovenLarge,ILD_min_21winSize_dirAroundnLarge,trainOrTest
1482751,NaN,NaN,NaN,NaN,359.00,NaN,0.014,NaN,NaN,NaN,61.724,NaN,NaN,NaN,53.940,NaN,NaN,NaN,NaN,NaN,NaN,0.191,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,00/10-35-081-15W4/0,154240,14000,13000,348.0,321.0,1,3,56.066128,-112.234008,300.5,323.5,23.0,325.0,13000,321.0,3,14000,348.0,1,-38.00,-11.00,0,0,34.00,False,False,140.0,360.0,219.00,1.00,220.0,FromBotWell,1.00,4,61.724,61.724,61.724,61.724,61.724,61.724,61.724,61.724,61.724,61.724,61.724,61.724,61.724,61.724,61.724,61.724,61.724,61.724,61.724,61.724,61.724,61.724,61.724,61.724,61.724,61.724,61.724,61.724,61.724,61.724,61.724,61.724,53.940,53.940,53.940,53.940,53.940,53.940,53.940,53.940,53.940,53.940,53.940,53.940,53.940,53.940,53.940,53.940,53.940,53.940,53.940,53.940,53.940,53.940,53.940,53.940,53.940,53.940,53.940,53.940,53.940,53.940,53.940,53.940,train
1482752,NaN,NaN,NaN,NaN,359.25,NaN,0.014,NaN,NaN,NaN,59.927,NaN,NaN,NaN,63.882,NaN,NaN,NaN,NaN,NaN,NaN,0.167,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,00/10-35-081-15W4/0,154240,14000,13000,348.0,321.0,1,3,56.066128,-112.234008,300.5,323.5,23.0,325.0,13000,321.0,3,14000,348.0,1,-38.25,-11.25,0,0,34.25,False,False,140.0,360.0,219.25,0.75,220.0,FromBotWell,0.75,3,59.927,59.927,59.927,59.927,59.927,59.927,59.927,59.927,59.927,59.927,59.927,59.927,59.927,59.927,59.927,59.927,59.927,59.927,59.927,59.927,59.927,59.927,59.927,59.927,59.927,59.92

In [24]:
df_test_5 = df_all_Col_preSplit_wTrainTest.copy()

### Additionally, we want to split such:
- balance class sizes somewhat otherwise rows not near the pick will dominate the population

- create new dataframe that keeps every row that is classed at or near the pick but for rows outside that area only keeps a some fraction based on remainder after division?

In [25]:
def countRowsByClassOfNearPickOrNot(df,arrayOfClass,divisionInt,classToShrink):
    for eachClass in arrayOfClass:
        print("length of rows with "+str(eachClass)+" in cat_isTopMcMrNearby_known:",len(df[df['cat_isTopMcMrNearby_known'] == eachClass]))
    df_NearPickZeroSmall = df.loc[(df.index%10 != 3) & (df['cat_isTopMcMrNearby_known'] == classToShrink)]
    print("length of rows with 0 in cat_isTopMcMrNearby_known and %"+str(divisionInt)+" == 0 is:",len(df_NearPickZeroSmall))
    print("% reduction in classs 0 is:", math.floor(len(df_NearPickZeroSmall) / len(df['cat_isTopMcMrNearby_known'] == classToShrink) * 100),"%")
    total_after_reduction_in_bigger_class = len(df[df['cat_isTopMcMrNearby_known'] == classToShrink]) -len(df_NearPickZeroSmall)
    print("if taken out using this remainder, the total number of 0 class will be: ",total_after_reduction_in_bigger_class)
#     print("ratio between that class away from pick and classes near pick is :":)
    return df_NearPickZeroSmall

In [26]:
class_array_NearPick = [100,95,60,0]
test_df_return = countRowsByClassOfNearPickOrNot(df_test_5,class_array_NearPick,2,0)
# test_df_return[0:200]['DEPT']

length of rows with 100 in cat_isTopMcMrNearby_known: 1014
length of rows with 95 in cat_isTopMcMrNearby_known: 4819
length of rows with 60 in cat_isTopMcMrNearby_known: 61345
length of rows with 0 in cat_isTopMcMrNearby_known: 1415578
length of rows with 0 in cat_isTopMcMrNearby_known and %2 == 0 is: 1274047
% reduction in classs 0 is: 85 %
if taken out using this remainder, the total number of 0 class will be:  141531


In [33]:
def dropsRowsWithMatchClassAndDeptRemainderIsZero(df,Col,RemainderInt,classToShrink):
    """
    Takes as input a dataframe, a column, a remainder integer, and a class within the column.
    Returns the dataframe minus the rows that match the ClassToShrink in the Col and have a depth from the DEPT col with a remainder of zero.
    """
    print("original lenght of dataframe = ",len(df))
    df_new = df.drop(df[(df[Col] == classToShrink) & (df.index%10 != 0)].index)
    print("length of new dataframe after dropping rows = ",len(df_new))
    print("number of rows dropped = ",len(df)-len(df_new))
    print("length of 0 class is :",len(df_new[df_new[Col] == classToShrink]))
    return df_new

In [34]:
df_all_Col_preSplit_wTrainTest_ClassBalanced = dropsRowsWithMatchClassAndDeptRemainderIsZero(df_all_Col_preSplit_wTrainTest,'cat_isTopMcMrNearby_known',7,0)

original lenght of dataframe =  1482756
length of new dataframe after dropping rows =  208758
number of rows dropped =  1273998
length of 0 class is : 141580


In [29]:
df_all_Col_preSplit_wTrainTest_ClassBalanced.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 208758 entries, 0 to 1482750
Columns: 132 entries, CALI to trainOrTest
dtypes: bool(2), float64(115), int64(12), object(3)
memory usage: 209.0+ MB


In [30]:
test = pd.DataFrame()
type(test)

pandas.core.frame.DataFrame

In [43]:
test.append(df_all_Col_preSplit_wTrainTest_ClassBalanced)
test.head()

""


In [49]:

test = df_all_Col_preSplit_wTrainTest_ClassBalanced.append(df_all_Col_preSplit_wTrainTest_ClassBalanced, ignore_index=True)
test.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 417516 entries, 0 to 417515
Columns: 132 entries, CALI to trainOrTest
dtypes: bool(2), float64(115), int64(12), object(3)
memory usage: 414.9+ MB


In [50]:
df_class100 = df_all_Col_preSplit_wTrainTest_ClassBalanced[df_all_Col_preSplit_wTrainTest_ClassBalanced['cat_isTopMcMrNearby_known'] == 100]

In [51]:
df_class100.head()

,CALI,COND,DELT,DENS,DEPT,DEPTH,DPHI,DPHI:1,DPHI:2,DT,GR,GR:1,GR:2,IL,ILD,ILD:1,ILD:2,ILM,LITH,LLD,LLS,NPHI,PHID,PHIN,RESD,RHOB,RT,SFL,SFLU,SN,SNP,SP,UWI,SitID,McMurray_Base_HorID,McMurray_Top_HorID,McMurray_Base_DEPTH,McMurray_Top_DEPTH,McMurray_Base_Qual,McMurray_Top_Qual,lat,lng,NN1_McMurray_Top_DEPTH,NN1_McMurray_Base_DEPTH,NN1_thickness,MM_Top_Depth_predBy_NN1thick,HorID,Pick,Quality,HorID_paleoz,Pick_paleoz,Quality_paleoz,diff_TMcM_Pick_v_DEPT,diff_TPal_Pick_v_DEPT,cat_isTopMcMrNearby_known,cat_isTopPalNearby_known,DistFrom_NN1_TopDepth_Abs,NewWell,LastBitWell,TopWellDept,BotWellDept,FromTopWell,FromBotWell,WellThickness,closerToBotOrTop,closTopBotDist,rowsToEdge,GR_min_5winSize_dirAroundMin,GR_min_5winSize_dirAboveMin,GR_min_5winSize_dirAroundMax,GR_min_5winSize_dirAboveMax,GR_min_5winSize_dirAroundMean,GR_min_5winSize_dirAboveMean,GR_min_5winSize_dirAbovenLarge,GR_min_5winSize_dirAroundnLarge,GR_min_7winSize_dirAroundMin,GR_min_7winSize_dirAboveMin,GR_min_7winSize_dirAroundMax,GR_min_7winSize_dirAboveMax,GR_min_7winSize_dirAroundMean,GR_min_7winSize_dirAboveMean,GR_min_7winSize_dirAbovenLarge,GR_min_7winSize_dirAroundnLarge,GR_min_11winSize_dirAroundMin,GR_min_11winSize_dirAboveMin,GR_min_11winSize_dirAroundMax,GR_min_11winSize_dirAboveMax,GR_min_11winSize_dirAroundMean,GR_min_11winSize_dirAboveMean,GR_min_11winSize_dirAbovenLarge,GR_min_11winSize_dirAroundnLarge,GR_min_21winSize_dirAroundMin,GR_min_21winSize_dirAboveMin,GR_min_21winSize_dirAroundMax,GR_min_21winSize_dirAboveMax,GR_min_21winSize_dirAroundMean,GR_min_21winSize_dirAboveMean,GR_min_21winSize_dirAbovenLarge,GR_min_21winSize_dirAroundnLarge,ILD_min_5winSize_dirAroundMin,ILD_min_5winSize_dirAboveMin,ILD_min_5winSize_dirAroundMax,ILD_min_5winSize_dirAboveMax,ILD_min_5winSize_dirAroundMean,ILD_min_5winSize_dirAboveMean,ILD_min_5winSize_dirAbovenLarge,ILD_min_5winSize_dirAroundnLarge,ILD_min_7winSize_dirAroundMin,ILD_min_7winSize_dirAboveMin,ILD_min_7winSize_dirAroundMax,ILD_min_7winSize_dirAboveMax,ILD_min_7winSize_dirAroundMean,ILD_min_7winSize_dirAboveMean,ILD_min_7winSize_dirAbovenLarge,ILD_min_7winSize_dirAroundnLarge,ILD_min_11winSize_dirAroundMin,ILD_min_11winSize_dirAboveMin,ILD_min_11winSize_dirAroundMax,ILD_min_11winSize_dirAboveMax,ILD_min_11winSize_dirAroundMean,ILD_min_11winSize_dirAboveMean,ILD_min_11winSize_dirAbovenLarge,ILD_min_11winSize_dirAroundnLarge,ILD_min_21winSize_dirAroundMin,ILD_min_21winSize_dirAboveMin,ILD_min_21winSize_dirAroundMax,ILD_min_21winSize_dirAboveMax,ILD_min_21winSize_dirAroundMean,ILD_min_21winSize_dirAboveMean,ILD_min_21winSize_dirAbovenLarge,ILD_min_21winSize_dirAroundnLarge,trainOrTest
4128,NaN,NaN,NaN,NaN,208.50,NaN,0.262,NaN,NaN,NaN,96.419,NaN,NaN,NaN,7.980,NaN,NaN,NaN,NaN,NaN,NaN,0.451,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,00/06-19-089-18W4/0,117839,14000,13000,217.0,208.50,1,2,56.731529,-112.866328,230.00,236.00,6.00,211.00,13000,208.50,2,14000,217.0,1,0.0,8.50,100,0,2.50,False,False,89.0,212.75,119.50,4.25,123.75,FromBotWell,4.25,17,54.389,96.419,98.636,96.419,83.4902,96.4190,96.4190,96.4190,49.040,96.419,98.636,96.419,76.282286,96.419000,96.4190,96.4190,96.419,96.419,96.419,96.419,96.419000,96.419000,96.4190,96.4190,96.419,96.419,96.419,96.419,96.419000,96.419000,96.4190,96.4190,6.415,7.980,15.270,7.980,9.5620,7.9800,7.9800,7.9800,6.415,7.980,20.165,7.980,10.773714,7.980000,7.9800,7.9800,7.980,7.980,7.980,7.980,7.980000,7.980000,7.9800,7.9800,7.980,7.980,7.980,7.980,7.980000,7.980000,7.9800,7.9800,train
6784,161.406,NaN,NaN,NaN,84.00,NaN,0.247,NaN,NaN,NaN,78.727,NaN,NaN,NaN,38.450,NaN,NaN,NaN,NaN,NaN,NaN,0.625,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,289.042,AA/07-29-093-13W4/0,144744,14000,13000,102.0,84.00,1,3,57.095396,-112.054745,68.88,105.16,36.28,65.72,13000,84.00,3,14000,102.0,1,0.0,18.00,100,0,18.28,False,False,20.0,103.00,64.00,19.00,83.00,FromBotWell,19.00,76,32.690,65.394,92.496,92.496,66.0424,79.6646,79.6646,66.0424,32.690,65.394,92.496,92.496,65.807571,81.152857,84.7898,76.5944,31.899,65.394,92.496,100

In [52]:
for each2 in range(3):
    print(each2)

0
1
2


In [53]:
def addsRowsToBalanceClasses(df,rangeFor100,rangeFor95):
    """
    asdfsg
    Copies the rows with labels that don't occur very much so they are a larger part of dataframe
    """
#     first = pd.DataFrame()
    df_class100 = df[df['cat_isTopMcMrNearby_known'] == 100]
    df_class95 = df[df['cat_isTopMcMrNearby_known'] == 95]
    for each1 in range(rangeFor100):
        print(each1)
        df = df.append(df_class100, ignore_index=True)
    for each2 in range(rangeFor95):
        print(each2)
        df = df.append(df_class95, ignore_index=True)
    return df

In [54]:
df_all_Col_preSplit_wTrainTest_ClassBalanced2 = addsRowsToBalanceClasses(df_all_Col_preSplit_wTrainTest_ClassBalanced,50,10)

0
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
37
38
39
40
41
42
43
44
45
46
47
48
49
0
1
2
3
4
5
6
7
8
9


In [55]:
len(df_all_Col_preSplit_wTrainTest_ClassBalanced2)

307648

In [56]:
df_all_Col_preSplit_wTrainTest_ClassBalanced2.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 307648 entries, 0 to 307647
Columns: 132 entries, CALI to trainOrTest
dtypes: bool(2), float64(115), int64(12), object(3)
memory usage: 305.7+ MB


In [57]:
df_all_Col_preSplit_wTrainTest_ClassBalanced = df_all_Col_preSplit_wTrainTest_ClassBalanced2

## Identify which columns to use as features 

In [58]:
col_list = df_all_Col_preSplit_wTrainTest_ClassBalanced.columns
print(col_list)

Index(['CALI', 'COND', 'DELT', 'DENS', 'DEPT', 'DEPTH', 'DPHI', 'DPHI:1', 'DPHI:2', 'DT',
       ...
       'ILD_min_11winSize_dirAroundnLarge', 'ILD_min_21winSize_dirAroundMin', 'ILD_min_21winSize_dirAboveMin', 'ILD_min_21winSize_dirAroundMax', 'ILD_min_21winSize_dirAboveMax', 'ILD_min_21winSize_dirAroundMean', 'ILD_min_21winSize_dirAboveMean', 'ILD_min_21winSize_dirAbovenLarge', 'ILD_min_21winSize_dirAroundnLarge', 'trainOrTest'], dtype='object', length=132)


In [59]:
df_all_Col_preSplit_wTrainTest_ClassBalanced.head()

,CALI,COND,DELT,DENS,DEPT,DEPTH,DPHI,DPHI:1,DPHI:2,DT,GR,GR:1,GR:2,IL,ILD,ILD:1,ILD:2,ILM,LITH,LLD,LLS,NPHI,PHID,PHIN,RESD,RHOB,RT,SFL,SFLU,SN,SNP,SP,UWI,SitID,McMurray_Base_HorID,McMurray_Top_HorID,McMurray_Base_DEPTH,McMurray_Top_DEPTH,McMurray_Base_Qual,McMurray_Top_Qual,lat,lng,NN1_McMurray_Top_DEPTH,NN1_McMurray_Base_DEPTH,NN1_thickness,MM_Top_Depth_predBy_NN1thick,HorID,Pick,Quality,HorID_paleoz,Pick_paleoz,Quality_paleoz,diff_TMcM_Pick_v_DEPT,diff_TPal_Pick_v_DEPT,cat_isTopMcMrNearby_known,cat_isTopPalNearby_known,DistFrom_NN1_TopDepth_Abs,NewWell,LastBitWell,TopWellDept,BotWellDept,FromTopWell,FromBotWell,WellThickness,closerToBotOrTop,closTopBotDist,rowsToEdge,GR_min_5winSize_dirAroundMin,GR_min_5winSize_dirAboveMin,GR_min_5winSize_dirAroundMax,GR_min_5winSize_dirAboveMax,GR_min_5winSize_dirAroundMean,GR_min_5winSize_dirAboveMean,GR_min_5winSize_dirAbovenLarge,GR_min_5winSize_dirAroundnLarge,GR_min_7winSize_dirAroundMin,GR_min_7winSize_dirAboveMin,GR_min_7winSize_dirAroundMax,GR_min_7winSize_dirAboveMax,GR_min_7winSize_dirAroundMean,GR_min_7winSize_dirAboveMean,GR_min_7winSize_dirAbovenLarge,GR_min_7winSize_dirAroundnLarge,GR_min_11winSize_dirAroundMin,GR_min_11winSize_dirAboveMin,GR_min_11winSize_dirAroundMax,GR_min_11winSize_dirAboveMax,GR_min_11winSize_dirAroundMean,GR_min_11winSize_dirAboveMean,GR_min_11winSize_dirAbovenLarge,GR_min_11winSize_dirAroundnLarge,GR_min_21winSize_dirAroundMin,GR_min_21winSize_dirAboveMin,GR_min_21winSize_dirAroundMax,GR_min_21winSize_dirAboveMax,GR_min_21winSize_dirAroundMean,GR_min_21winSize_dirAboveMean,GR_min_21winSize_dirAbovenLarge,GR_min_21winSize_dirAroundnLarge,ILD_min_5winSize_dirAroundMin,ILD_min_5winSize_dirAboveMin,ILD_min_5winSize_dirAroundMax,ILD_min_5winSize_dirAboveMax,ILD_min_5winSize_dirAroundMean,ILD_min_5winSize_dirAboveMean,ILD_min_5winSize_dirAbovenLarge,ILD_min_5winSize_dirAroundnLarge,ILD_min_7winSize_dirAroundMin,ILD_min_7winSize_dirAboveMin,ILD_min_7winSize_dirAroundMax,ILD_min_7winSize_dirAboveMax,ILD_min_7winSize_dirAroundMean,ILD_min_7winSize_dirAboveMean,ILD_min_7winSize_dirAbovenLarge,ILD_min_7winSize_dirAroundnLarge,ILD_min_11winSize_dirAroundMin,ILD_min_11winSize_dirAboveMin,ILD_min_11winSize_dirAroundMax,ILD_min_11winSize_dirAboveMax,ILD_min_11winSize_dirAroundMean,ILD_min_11winSize_dirAboveMean,ILD_min_11winSize_dirAbovenLarge,ILD_min_11winSize_dirAroundnLarge,ILD_min_21winSize_dirAroundMin,ILD_min_21winSize_dirAboveMin,ILD_min_21winSize_dirAroundMax,ILD_min_21winSize_dirAboveMax,ILD_min_21winSize_dirAroundMean,ILD_min_21winSize_dirAboveMean,ILD_min_21winSize_dirAbovenLarge,ILD_min_21winSize_dirAroundnLarge,trainOrTest
0,167.003,NaN,NaN,NaN,149.602,NaN,0.227,NaN,NaN,NaN,102.473,NaN,NaN,NaN,0.000,NaN,NaN,NaN,NaN,NaN,NaN,0.460,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,00/10-32-080-20W4/0,112385,14000,13000,384.66,377.95,1,3,55.978836,-113.095365,389.0,414.0,25.0,359.66,13000,377.95,3,14000,384.66,1,228.348,235.058,0,0,210.058,True,False,149.602,396.102,0.0,246.5,246.5,FromTopWell,0.0,0,102.473,102.473,102.473,102.473,102.4730,102.4730,102.4730,102.4730,102.473,102.473,102.473,102.473,102.473000,102.473000,102.4730,102.4730,102.473,102.473,102.473,102.473,102.473000,102.473,102.473,102.473,102.473,102.473,102.473,102.473,102.473,102.473,102.473,102.473,0.000,0.000,0.000,0.000,0.0000,0.0000,0.0000,0.0000,0.000,0.000,0.000,0.000,0.000000,0.000000,0.0000,0.0000,0.000,0.000,0.000,0.000,0.000000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,train
1,166.675,NaN,NaN,NaN,152.102,NaN,0.269,NaN,NaN,NaN,26.625,NaN,NaN,NaN,30.179,NaN,NaN,NaN,NaN,NaN,NaN,0.355,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,00/10-32-080-20W4/0,112385,14000,13000,384.66,377.95,1,3,55.978836,-113.095365,389.0,414.0,25.0,359.66,13000,377.95,3,14000,384.66,1,225.848,232.558,0,0,207.558,False,False,149.602,396.102,2.5,244.0,246.5,FromTopWell,2.5,10,25.825,26.625,50.213,26.625,32.7680,26.6250,26.6250,26.6250,26.625,26.625,26.625,26.625,26.625000,26.625000,26.6250,26.6250,26.625,2

In [60]:
col_list = list(col_list)
col_list

['CALI',
 'COND',
 'DELT',
 'DENS',
 'DEPT',
 'DEPTH',
 'DPHI',
 'DPHI:1',
 'DPHI:2',
 'DT',
 'GR',
 'GR:1',
 'GR:2',
 'IL',
 'ILD',
 'ILD:1',
 'ILD:2',
 'ILM',
 'LITH',
 'LLD',
 'LLS',
 'NPHI',
 'PHID',
 'PHIN',
 'RESD',
 'RHOB',
 'RT',
 'SFL',
 'SFLU',
 'SN',
 'SNP',
 'SP',
 'UWI',
 'SitID',
 'McMurray_Base_HorID',
 'McMurray_Top_HorID',
 'McMurray_Base_DEPTH',
 'McMurray_Top_DEPTH',
 'McMurray_Base_Qual',
 'McMurray_Top_Qual',
 'lat',
 'lng',
 'NN1_McMurray_Top_DEPTH',
 'NN1_McMurray_Base_DEPTH',
 'NN1_thickness',
 'MM_Top_Depth_predBy_NN1thick',
 'HorID',
 'Pick',
 'Quality',
 'HorID_paleoz',
 'Pick_paleoz',
 'Quality_paleoz',
 'diff_TMcM_Pick_v_DEPT',
 'diff_TPal_Pick_v_DEPT',
 'cat_isTopMcMrNearby_known',
 'cat_isTopPalNearby_known',
 'DistFrom_NN1_TopDepth_Abs',
 'NewWell',
 'LastBitWell',
 'TopWellDept',
 'BotWellDept',
 'FromTopWell',
 'FromBotWell',
 'WellThickness',
 'closerToBotOrTop',
 'closTopBotDist',
 'rowsToEdge',
 'GR_min_5winSize_dirAroundMin',
 'GR_min_5winSize_dirA

In [61]:
## NOTE WE ARE LEAVING THE UWI in for now but will take it out after dataframe is split into train/test portions!!!!
train_feat_bigList = [
 'UWI',
 'trainOrTest',
 'CALI',
 'COND',
 'DELT',
 'DENS',
 'DPHI',
 'DPHI:1',
 'DPHI:2',
 'DT',
 'GR',
 'GR:1',
 'GR:2',
 'IL',
 'ILD',
 'ILD:1',
 'ILD:2',
 'ILM',
 'LITH',
 'LLD',
 'LLS',
 'NPHI',
 'PHID',
 'PHIN',
 'RESD',
 'RHOB',
 'RT',
 'SFL',
 'SFLU',
 'SN',
 'SNP',
 'SP',
 'McMurray_Base_Qual',
 'McMurray_Top_Qual',
 'lat',
 'lng',  
 'NN1_thickness',
 'MM_Top_Depth_predBy_NN1thick',
 'Quality',
 'Quality_paleoz',
 'DistFrom_NN1_TopDepth_Abs',
 'BotWellDept',
 'FromTopWell',
 'FromBotWell',
 'WellThickness',
 'rowsToEdge',
 'GR_min_5winSize_dirAroundMin',
 'GR_min_5winSize_dirAboveMin',
 'GR_min_5winSize_dirAroundMax',
 'GR_min_5winSize_dirAboveMax',
 'GR_min_5winSize_dirAroundMean',
 'GR_min_5winSize_dirAboveMean',
 'GR_min_5winSize_dirAbovenLarge',
 'GR_min_5winSize_dirAroundnLarge',
 'GR_min_7winSize_dirAroundMin',
 'GR_min_7winSize_dirAboveMin',
 'GR_min_7winSize_dirAroundMax',
 'GR_min_7winSize_dirAboveMax',
 'GR_min_7winSize_dirAroundMean',
 'GR_min_7winSize_dirAboveMean',
 'GR_min_7winSize_dirAbovenLarge',
 'GR_min_7winSize_dirAroundnLarge',
 'GR_min_11winSize_dirAroundMin',
 'GR_min_11winSize_dirAboveMin',
 'GR_min_11winSize_dirAroundMax',
 'GR_min_11winSize_dirAboveMax',
 'GR_min_11winSize_dirAroundMean',
 'GR_min_11winSize_dirAboveMean',
 'GR_min_11winSize_dirAbovenLarge',
 'GR_min_11winSize_dirAroundnLarge',
 'GR_min_21winSize_dirAroundMin',
 'GR_min_21winSize_dirAboveMin',
 'GR_min_21winSize_dirAroundMax',
 'GR_min_21winSize_dirAboveMax',
 'GR_min_21winSize_dirAroundMean',
 'GR_min_21winSize_dirAboveMean',
 'GR_min_21winSize_dirAbovenLarge',
 'GR_min_21winSize_dirAroundnLarge',
 'ILD_min_5winSize_dirAroundMin',
 'ILD_min_5winSize_dirAboveMin',
 'ILD_min_5winSize_dirAroundMax',
 'ILD_min_5winSize_dirAboveMax',
 'ILD_min_5winSize_dirAroundMean',
 'ILD_min_5winSize_dirAboveMean',
 'ILD_min_5winSize_dirAbovenLarge',
 'ILD_min_5winSize_dirAroundnLarge',
 'ILD_min_7winSize_dirAroundMin',
 'ILD_min_7winSize_dirAboveMin',
 'ILD_min_7winSize_dirAroundMax',
 'ILD_min_7winSize_dirAboveMax',
 'ILD_min_7winSize_dirAroundMean',
 'ILD_min_7winSize_dirAboveMean',
 'ILD_min_7winSize_dirAbovenLarge',
 'ILD_min_7winSize_dirAroundnLarge',
 'ILD_min_11winSize_dirAroundMin',
 'ILD_min_11winSize_dirAboveMin',
 'ILD_min_11winSize_dirAroundMax',
 'ILD_min_11winSize_dirAboveMax',
 'ILD_min_11winSize_dirAroundMean',
 'ILD_min_11winSize_dirAboveMean',
 'ILD_min_11winSize_dirAbovenLarge',
 'ILD_min_11winSize_dirAroundnLarge',
 'ILD_min_21winSize_dirAroundMin',
 'ILD_min_21winSize_dirAboveMin',
 'ILD_min_21winSize_dirAroundMax',
 'ILD_min_21winSize_dirAboveMax',
 'ILD_min_21winSize_dirAroundMean',
 'ILD_min_21winSize_dirAboveMean',
 'ILD_min_21winSize_dirAbovenLarge',
 'ILD_min_21winSize_dirAroundnLarge']

In [62]:
len(train_feat_bigList)

110

In [79]:
takeOutColList = [
    'FromBotWell',
    'FromTopWel'
    'MM_Top_Depth_predBy_NN1thick',
    'rowsToEdge',
    'Quality',
    'McMurray_Base_Qual',
     'McMurray_Top_Qual',
     'lat',
     'lng',    
]

In [80]:
df_train_feat = df_all_Col_preSplit_wTrainTest_ClassBalanced[train_feat_bigList]

In [81]:
df_train_feat.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 307648 entries, 0 to 307647
Columns: 110 entries, UWI to ILD_min_21winSize_dirAroundnLarge
dtypes: float64(103), int64(5), object(2)
memory usage: 258.2+ MB


In [82]:
df_train_feat.head()

,UWI,trainOrTest,CALI,COND,DELT,DENS,DPHI,DPHI:1,DPHI:2,DT,GR,GR:1,GR:2,IL,ILD,ILD:1,ILD:2,ILM,LITH,LLD,LLS,NPHI,PHID,PHIN,RESD,RHOB,RT,SFL,SFLU,SN,SNP,SP,McMurray_Base_Qual,McMurray_Top_Qual,lat,lng,NN1_thickness,MM_Top_Depth_predBy_NN1thick,Quality,Quality_paleoz,DistFrom_NN1_TopDepth_Abs,BotWellDept,FromTopWell,FromBotWell,WellThickness,rowsToEdge,GR_min_5winSize_dirAroundMin,GR_min_5winSize_dirAboveMin,GR_min_5winSize_dirAroundMax,GR_min_5winSize_dirAboveMax,GR_min_5winSize_dirAroundMean,GR_min_5winSize_dirAboveMean,GR_min_5winSize_dirAbovenLarge,GR_min_5winSize_dirAroundnLarge,GR_min_7winSize_dirAroundMin,GR_min_7winSize_dirAboveMin,GR_min_7winSize_dirAroundMax,GR_min_7winSize_dirAboveMax,GR_min_7winSize_dirAroundMean,GR_min_7winSize_dirAboveMean,GR_min_7winSize_dirAbovenLarge,GR_min_7winSize_dirAroundnLarge,GR_min_11winSize_dirAroundMin,GR_min_11winSize_dirAboveMin,GR_min_11winSize_dirAroundMax,GR_min_11winSize_dirAboveMax,GR_min_11winSize_dirAroundMean,GR_min_11winSize_dirAboveMean,GR_min_11winSize_dirAbovenLarge,GR_min_11winSize_dirAroundnLarge,GR_min_21winSize_dirAroundMin,GR_min_21winSize_dirAboveMin,GR_min_21winSize_dirAroundMax,GR_min_21winSize_dirAboveMax,GR_min_21winSize_dirAroundMean,GR_min_21winSize_dirAboveMean,GR_min_21winSize_dirAbovenLarge,GR_min_21winSize_dirAroundnLarge,ILD_min_5winSize_dirAroundMin,ILD_min_5winSize_dirAboveMin,ILD_min_5winSize_dirAroundMax,ILD_min_5winSize_dirAboveMax,ILD_min_5winSize_dirAroundMean,ILD_min_5winSize_dirAboveMean,ILD_min_5winSize_dirAbovenLarge,ILD_min_5winSize_dirAroundnLarge,ILD_min_7winSize_dirAroundMin,ILD_min_7winSize_dirAboveMin,ILD_min_7winSize_dirAroundMax,ILD_min_7winSize_dirAboveMax,ILD_min_7winSize_dirAroundMean,ILD_min_7winSize_dirAboveMean,ILD_min_7winSize_dirAbovenLarge,ILD_min_7winSize_dirAroundnLarge,ILD_min_11winSize_dirAroundMin,ILD_min_11winSize_dirAboveMin,ILD_min_11winSize_dirAroundMax,ILD_min_11winSize_dirAboveMax,ILD_min_11winSize_dirAroundMean,ILD_min_11winSize_dirAboveMean,ILD_min_11winSize_dirAbovenLarge,ILD_min_11winSize_dirAroundnLarge,ILD_min_21winSize_dirAroundMin,ILD_min_21winSize_dirAboveMin,ILD_min_21winSize_dirAroundMax,ILD_min_21winSize_dirAboveMax,ILD_min_21winSize_dirAroundMean,ILD_min_21winSize_dirAboveMean,ILD_min_21winSize_dirAbovenLarge,ILD_min_21winSize_dirAroundnLarge
0,00/10-32-080-20W4/0,train,167.003,NaN,NaN,NaN,0.227,NaN,NaN,NaN,102.473,NaN,NaN,NaN,0.000,NaN,NaN,NaN,NaN,NaN,NaN,0.460,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1,3,55.978836,-113.095365,25.0,359.66,3,1,210.058,396.102,0.0,246.5,246.5,0,102.473,102.473,102.473,102.473,102.4730,102.4730,102.4730,102.4730,102.473,102.473,102.473,102.473,102.473000,102.473000,102.4730,102.4730,102.473,102.473,102.473,102.473,102.473000,102.473,102.473,102.473,102.473,102.473,102.473,102.473,102.473,102.473,102.473,102.473,0.000,0.000,0.000,0.000,0.0000,0.0000,0.0000,0.0000,0.000,0.000,0.000,0.000,0.000000,0.000000,0.0000,0.0000,0.000,0.000,0.000,0.000,0.000000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000
1,00/10-32-080-20W4/0,train,166.675,NaN,NaN,NaN,0.269,NaN,NaN,NaN,26.625,NaN,NaN,NaN,30.179,NaN,NaN,NaN,NaN,NaN,NaN,0.355,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1,3,55.978836,-113.095365,25.0,359.66,3,1,207.558,396.102,2.5,244.0,246.5,10,25.825,26.625,50.213,26.625,32.7680,26.6250,26.6250,26.6250,26.625,26.625,26.625,26.625,26.625000,26.625000,26.6250,26.6250,26.625,26.625,26.625,26.625,26.625000,26.625,26.625,26.625,26.625,26.625,26.625,26.625,26.625,26.625,26.625,26.625,20.262,30.179,30.370,30.179,27.1088,30.1790,30.1790,30.1790,30.179,30.179,30.179,30.179,30.179000,30.179000,30.1790,30.1790,30.179,30.179,30.179,30.179,30.179000,30.179,30.179,30.179,30.179,30.179,30.179,30.179,30.179,30.179,30.179,30.179
2,00/10-32-080-20W4/0,train,211.701,NaN,NaN,NaN,0.339,NaN,NaN,NaN,31.562,NaN,NaN,NaN,21.793,NaN,NaN,NaN,NaN,NaN,NaN,0.428,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1,3,55.978836,-113.095365,25.0,359.66,3,1,205.058,396.102,5.0,241.5,246.5,20,23.605,31.562,49.258,31

In [91]:
training_feats_w_lowCount = ['RHOB','SP','CALI','COND','DELT','DENS','DPHI:1','DPHI:2','DT','GR:1','GR:2','IL','ILD:1','ILD:2','ILM','LITH','LLD','LLS','PHID','PHIN','RESD','RT','SFL','SFLU','SN','SNP','Sp']

In [92]:
train_feats_minusLowCount = [x for x in train_feat_bigList if x not in training_feats_w_lowCount]

In [93]:
train_feats_minusLowCount = [x for x in train_feats_minusLowCount if x not in takeOutColList]

In [95]:
df_train_featWithHighCount = df_train_feat[train_feats_minusLowCount]

In [97]:
len(train_feats_minusLowCount)

77

In [98]:
df_train_featWithHighCount.describe()

,DPHI,GR,ILD,NPHI,NN1_thickness,Quality_paleoz,DistFrom_NN1_TopDepth_Abs,BotWellDept,FromTopWell,FromBotWell,WellThickness,GR_min_5winSize_dirAroundMin,GR_min_5winSize_dirAboveMin,GR_min_5winSize_dirAroundMax,GR_min_5winSize_dirAboveMax,GR_min_5winSize_dirAroundMean,GR_min_5winSize_dirAboveMean,GR_min_5winSize_dirAbovenLarge,GR_min_5winSize_dirAroundnLarge,GR_min_7winSize_dirAroundMin,GR_min_7winSize_dirAboveMin,GR_min_7winSize_dirAroundMax,GR_min_7winSize_dirAboveMax,GR_min_7winSize_dirAroundMean,GR_min_7winSize_dirAboveMean,GR_min_7winSize_dirAbovenLarge,GR_min_7winSize_dirAroundnLarge,GR_min_11winSize_dirAroundMin,GR_min_11winSize_dirAboveMin,GR_min_11winSize_dirAroundMax,GR_min_11winSize_dirAboveMax,GR_min_11winSize_dirAroundMean,GR_min_11winSize_dirAboveMean,GR_min_11winSize_dirAbovenLarge,GR_min_11winSize_dirAroundnLarge,GR_min_21winSize_dirAroundMin,GR_min_21winSize_dirAboveMin,GR_min_21winSize_dirAroundMax,GR_min_21winSize_dirAboveMax,GR_min_21winSize_dirAroundMean,GR_min_21winSize_dirAboveMean,GR_min_21winSize_dirAbovenLarge,GR_min_21winSize_dirAroundnLarge,ILD_min_5winSize_dirAroundMin,ILD_min_5winSize_dirAboveMin,ILD_min_5winSize_dirAroundMax,ILD_min_5winSize_dirAboveMax,ILD_min_5winSize_dirAroundMean,ILD_min_5winSize_dirAboveMean,ILD_min_5winSize_dirAbovenLarge,ILD_min_5winSize_dirAroundnLarge,ILD_min_7winSize_dirAroundMin,ILD_min_7winSize_dirAboveMin,ILD_min_7winSize_dirAroundMax,ILD_min_7winSize_dirAboveMax,ILD_min_7winSize_dirAroundMean,ILD_min_7winSize_dirAboveMean,ILD_min_7winSize_dirAbovenLarge,ILD_min_7winSize_dirAroundnLarge,ILD_min_11winSize_dirAroundMin,ILD_min_11winSize_dirAboveMin,ILD_min_11winSize_dirAroundMax,ILD_min_11winSize_dirAboveMax,ILD_min_11winSize_dirAroundMean,ILD_min_11winSize_dirAboveMean,ILD_min_11winSize_dirAbovenLarge,ILD_min_11winSize_dirAroundnLarge,ILD_min_21winSize_dirAroundMin,ILD_min_21winSize_dirAboveMin,ILD_min_21winSize_dirAroundMax,ILD_min_21winSize_dirAboveMax,ILD_min_21winSize_dirAroundMean,ILD_min_21winSize_dirAboveMean,ILD_min_21winSize_dirAbovenLarge,ILD_min_21winSize_dirAroundnLarge
count,279301.000000,307214.000000,305061.000000,289830.000000,306894.000000,307648.000000,306894.000000,307648.000000,307648.000000,307648.000000,307648.000000,307211.000000,307214.000000,307211.000000,307214.000000,307211.000000,307214.000000,307214.000000,307213.000000,307212.000000,307214.000000,307212.000000,307214.000000,307212.000000,307214.000000,307214.000000,307213.000000,307213.000000,307213.000000,307213.000000,307213.000000,307213.000000,307213.000000,307213.000000,307214.000000,307213.000000,307212.00000,307213.000000,307212.000000,307213.000000,307212.000000,307212.000000,307213.000000,305061.000000,305059.000000,305061.000000,305059.000000,305061.000000,305059.000000,305059.000000,305061.000000,305060.000000,305058.000000,305060.000000,305058.000000,305060.000000,305058.000000,305058.000000,305060.000000,305059.000000,305058.000000,305059.000000,305058.000000,305059.000000,305058.000000,305058.000000,305059.000000,305057.000000,305057.000000,305057.000000,305057.000000,305057.000000,305057.000000,305057.000000,305058.000000
mean,0.251156,79.313089,23.539498,0.416785,38.724685,1.333326,53.485277,470.899040,138.605100,82.252457,220.857556,71.573041,71.984871,86.120706,84.991337,78.972551,78.517582,78.517582,78.973720,69.013426,69.910424,88.065090,86.434257,78.734864,78.087746,80.821154,81.849248,65.582260,67.632727,90.750929,88.581815,78.329950,77.784944,83.872589,85.310916,61.739544,66.41741,94.620626,91.347238,78.004253,78.507993,88.076699,88.620342,18.918723,17.821049,30.749279,29.813357,23.988587,23.612654,23.612654,23.971850,17.427300,16.606689,34.325458,31.405646,24.307483,23.335367,25.728592,26.420769,14.762723,15.276471,39.864485,33.327836,24.647141,22.711060,28.084316,30.630660,12.963767,15.902670,47.337650,35.854583,25.116095,23.140083,31.582544,36.048660
std,0.539236,24.889367,245.881390,0.740066,25.972044,0.770345,101.181928,195.932171,78.213462,82.747857,90.783465,23

-----------------

## Identify which columns to use as labels

#### The column 'cat_isTopMcMrNearby_known' is what we'll use as labels.
##### 100 = exactly the Top McMurray Pick
##### 95 if the distance between that depth and the Top McMurray Pick is -0.5 < x and x <0.5
##### 60 if the distance between that depth and the Top McMurray Pick is -5 < x and x < 5
##### 0 = not near the Top McMurray Pick
`df_all_wells_wKNN_DEPTHtoDEPT['cat_isTopMcMrNearby_known']=df_all_wells_wKNN_DEPTHtoDEPT['diff_TMcM_Pick_v_DEPT'].apply(lambda x: 100 if x==0 else ( 95 if (-0.5 < x and x <0.5) else 60 if (-5 < x and x <5) else 0))`

In [99]:
df_all_Col_preSplit_wTrainTest_ClassBalanced['cat_isTopMcMrNearby_known'].unique()

array([  0,  60,  95, 100])

In [100]:
labels = df_all_Col_preSplit_wTrainTest_ClassBalanced[['cat_isTopMcMrNearby_known','UWI','trainOrTest']]

In [101]:
labels.head()

,cat_isTopMcMrNearby_known,UWI,trainOrTest
0,0,00/10-32-080-20W4/0,train
1,0,00/10-32-080-20W4/0,train
2,0,00/10-32-080-20W4/0,train
3,0,00/10-32-080-20W4/0,train
4,0,00/10-32-080-20W4/0,train


In [102]:
labels.tail()

,cat_isTopMcMrNearby_known,UWI,trainOrTest
307643,95,00/11-18-079-03W5/0,train
307644,95,00/11-18-079-03W5/0,train
307645,95,00/11-18-079-03W5/0,train
307646,95,00/10-35-081-15W4/0,train
307647,95,00/10-35-081-15W4/0,train


In [103]:
len(labels)

307648

-----------------

## Now separate into 4 dataframes for train_labels, train_feat, test_labels, test_feat
## Then take off UWI and TrainTest col?

In [104]:
# labels_train = labels[labels['trainOrTest'] == 'train' ].drop(columns=['UWI', 'trainOrTest'])
# labels_test = labels[labels['trainOrTest'] == 'test' ].drop(columns=['UWI', 'trainOrTest'])

In [105]:
labels_train = labels[labels['trainOrTest'] == 'train' ]
labels_train = labels_train['cat_isTopMcMrNearby_known']
labels_test = labels[labels['trainOrTest'] == 'test' ]
labels_test = labels_test['cat_isTopMcMrNearby_known']

In [106]:
df_train_featWithHighCount_train = df_train_featWithHighCount[df_train_featWithHighCount['trainOrTest'] == 'train' ].drop(columns=['UWI', 'trainOrTest'])
df_train_featWithHighCount_test = df_train_featWithHighCount[df_train_featWithHighCount['trainOrTest'] == 'test' ].drop(columns=['UWI', 'trainOrTest'])

In [107]:
train_X = df_train_featWithHighCount_train
train_y = labels_train
test_X = df_train_featWithHighCount_test
test_y = labels_test

In [108]:
print(len(train_X))
train_X.head()

246352


,DPHI,GR,ILD,NPHI,NN1_thickness,Quality_paleoz,DistFrom_NN1_TopDepth_Abs,BotWellDept,FromTopWell,FromBotWell,WellThickness,GR_min_5winSize_dirAroundMin,GR_min_5winSize_dirAboveMin,GR_min_5winSize_dirAroundMax,GR_min_5winSize_dirAboveMax,GR_min_5winSize_dirAroundMean,GR_min_5winSize_dirAboveMean,GR_min_5winSize_dirAbovenLarge,GR_min_5winSize_dirAroundnLarge,GR_min_7winSize_dirAroundMin,GR_min_7winSize_dirAboveMin,GR_min_7winSize_dirAroundMax,GR_min_7winSize_dirAboveMax,GR_min_7winSize_dirAroundMean,GR_min_7winSize_dirAboveMean,GR_min_7winSize_dirAbovenLarge,GR_min_7winSize_dirAroundnLarge,GR_min_11winSize_dirAroundMin,GR_min_11winSize_dirAboveMin,GR_min_11winSize_dirAroundMax,GR_min_11winSize_dirAboveMax,GR_min_11winSize_dirAroundMean,GR_min_11winSize_dirAboveMean,GR_min_11winSize_dirAbovenLarge,GR_min_11winSize_dirAroundnLarge,GR_min_21winSize_dirAroundMin,GR_min_21winSize_dirAboveMin,GR_min_21winSize_dirAroundMax,GR_min_21winSize_dirAboveMax,GR_min_21winSize_dirAroundMean,GR_min_21winSize_dirAboveMean,GR_min_21winSize_dirAbovenLarge,GR_min_21winSize_dirAroundnLarge,ILD_min_5winSize_dirAroundMin,ILD_min_5winSize_dirAboveMin,ILD_min_5winSize_dirAroundMax,ILD_min_5winSize_dirAboveMax,ILD_min_5winSize_dirAroundMean,ILD_min_5winSize_dirAboveMean,ILD_min_5winSize_dirAbovenLarge,ILD_min_5winSize_dirAroundnLarge,ILD_min_7winSize_dirAroundMin,ILD_min_7winSize_dirAboveMin,ILD_min_7winSize_dirAroundMax,ILD_min_7winSize_dirAboveMax,ILD_min_7winSize_dirAroundMean,ILD_min_7winSize_dirAboveMean,ILD_min_7winSize_dirAbovenLarge,ILD_min_7winSize_dirAroundnLarge,ILD_min_11winSize_dirAroundMin,ILD_min_11winSize_dirAboveMin,ILD_min_11winSize_dirAroundMax,ILD_min_11winSize_dirAboveMax,ILD_min_11winSize_dirAroundMean,ILD_min_11winSize_dirAboveMean,ILD_min_11winSize_dirAbovenLarge,ILD_min_11winSize_dirAroundnLarge,ILD_min_21winSize_dirAroundMin,ILD_min_21winSize_dirAboveMin,ILD_min_21winSize_dirAroundMax,ILD_min_21winSize_dirAboveMax,ILD_min_21winSize_dirAroundMean,ILD_min_21winSize_dirAboveMean,ILD_min_21winSize_dirAbovenLarge,ILD_min_21winSize_dirAroundnLarge
0,0.227,102.473,0.000,0.460,25.0,1,210.058,396.102,0.0,246.5,246.5,102.473,102.473,102.473,102.473,102.4730,102.4730,102.4730,102.4730,102.473,102.473,102.473,102.473,102.473000,102.473000,102.4730,102.4730,102.473,102.473,102.473,102.473,102.473000,102.473,102.473,102.473,102.473,102.473,102.473,102.473,102.473,102.473,102.473,102.473,0.000,0.000,0.000,0.000,0.0000,0.0000,0.0000,0.0000,0.000,0.000,0.000,0.000,0.000000,0.000000,0.0000,0.0000,0.000,0.000,0.000,0.000,0.000000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000
1,0.269,26.625,30.179,0.355,25.0,1,207.558,396.102,2.5,244.0,246.5,25.825,26.625,50.213,26.625,32.7680,26.6250,26.6250,26.6250,26.625,26.625,26.625,26.625,26.625000,26.625000,26.6250,26.6250,26.625,26.625,26.625,26.625,26.625000,26.625,26.625,26.625,26.625,26.625,26.625,26.625,26.625,26.625,26.625,26.625,20.262,30.179,30.370,30.179,27.1088,30.1790,30.1790,30.1790,30.179,30.179,30.179,30.179,30.179000,30.179000,30.1790,30.1790,30.179,30.179,30.179,30.179,30.179000,30.179,30.179,30.179,30.179,30.179,30.179,30.179,30.179,30.179,30.179,30.179
2,0.339,31.562,21.793,0.428,25.0,1,205.058,396.102,5.0,241.5,246.5,23.605,31.562,49.258,31.562,34.0164,31.5620,31.5620,31.5620,22.700,31.562,60.528,31.562,36.187143,31.562000,31.5620,31.5620,31.562,31.562,31.562,31.562,31.562000,31.562,31.562,31.562,31.562,31.562,31.562,31.562,31.562,31.562,31.562,31.562,16.583,21.793,25.975,21.793,21.5708,21.7930,21.7930,21.7930,14.586,21.793,26.774,21.793,21.316286,21.793000,21.7930,21.7930,21.793,21.793,21.793,21.793,21.793000,21.793,21.793,21.793,21.793,21.793,21.793,21.793,21.793,21.793,21.793,21.793
3,0.291,51.257,7.449,0.452,25.0,1,202.558,396.102,7.5,239.0,246.5,40.739,37.621,72.481,51.257,53.8586,43.6546,43.6546,53.8586,37.621,37.621,87.074,60.965,56.284000,47.374143,50.6518,63.1256,37.621,51.257,88.401,51.257,59.979636,51.257,51.257,51.257,51.257,51.257,51.257,51.257,51.257,51.257

In [109]:
print(len(train_y))
train_y.head()

246352


0    0
1    0
2    0
3    0
4    0
Name: cat_isTopMcMrNearby_known, dtype: int64

In [110]:
print(len(test_X))
test_X.head()

61296


,DPHI,GR,ILD,NPHI,NN1_thickness,Quality_paleoz,DistFrom_NN1_TopDepth_Abs,BotWellDept,FromTopWell,FromBotWell,WellThickness,GR_min_5winSize_dirAroundMin,GR_min_5winSize_dirAboveMin,GR_min_5winSize_dirAroundMax,GR_min_5winSize_dirAboveMax,GR_min_5winSize_dirAroundMean,GR_min_5winSize_dirAboveMean,GR_min_5winSize_dirAbovenLarge,GR_min_5winSize_dirAroundnLarge,GR_min_7winSize_dirAroundMin,GR_min_7winSize_dirAboveMin,GR_min_7winSize_dirAroundMax,GR_min_7winSize_dirAboveMax,GR_min_7winSize_dirAroundMean,GR_min_7winSize_dirAboveMean,GR_min_7winSize_dirAbovenLarge,GR_min_7winSize_dirAroundnLarge,GR_min_11winSize_dirAroundMin,GR_min_11winSize_dirAboveMin,GR_min_11winSize_dirAroundMax,GR_min_11winSize_dirAboveMax,GR_min_11winSize_dirAroundMean,GR_min_11winSize_dirAboveMean,GR_min_11winSize_dirAbovenLarge,GR_min_11winSize_dirAroundnLarge,GR_min_21winSize_dirAroundMin,GR_min_21winSize_dirAboveMin,GR_min_21winSize_dirAroundMax,GR_min_21winSize_dirAboveMax,GR_min_21winSize_dirAroundMean,GR_min_21winSize_dirAboveMean,GR_min_21winSize_dirAbovenLarge,GR_min_21winSize_dirAroundnLarge,ILD_min_5winSize_dirAroundMin,ILD_min_5winSize_dirAboveMin,ILD_min_5winSize_dirAroundMax,ILD_min_5winSize_dirAboveMax,ILD_min_5winSize_dirAroundMean,ILD_min_5winSize_dirAboveMean,ILD_min_5winSize_dirAbovenLarge,ILD_min_5winSize_dirAroundnLarge,ILD_min_7winSize_dirAroundMin,ILD_min_7winSize_dirAboveMin,ILD_min_7winSize_dirAroundMax,ILD_min_7winSize_dirAboveMax,ILD_min_7winSize_dirAroundMean,ILD_min_7winSize_dirAboveMean,ILD_min_7winSize_dirAbovenLarge,ILD_min_7winSize_dirAroundnLarge,ILD_min_11winSize_dirAroundMin,ILD_min_11winSize_dirAboveMin,ILD_min_11winSize_dirAroundMax,ILD_min_11winSize_dirAboveMax,ILD_min_11winSize_dirAroundMean,ILD_min_11winSize_dirAboveMean,ILD_min_11winSize_dirAbovenLarge,ILD_min_11winSize_dirAroundnLarge,ILD_min_21winSize_dirAroundMin,ILD_min_21winSize_dirAboveMin,ILD_min_21winSize_dirAroundMax,ILD_min_21winSize_dirAboveMax,ILD_min_21winSize_dirAroundMean,ILD_min_21winSize_dirAboveMean,ILD_min_21winSize_dirAbovenLarge,ILD_min_21winSize_dirAroundnLarge
269,0.308,100.386,2.158,0.528,24.39,3,185.146,649.114,2.0,211.25,213.25,100.386,100.386,100.386,100.386,100.3860,100.3860,100.3860,100.3860,100.386,100.386,100.386,100.386,100.386000,100.386000,100.3860,100.3860,100.386,100.386,100.386,100.386,100.386000,100.386,100.3860,100.386,100.386,100.386,100.386,100.386,100.386000,100.386,100.386,100.386,2.158,2.158,2.158,2.158,2.1580,2.1580,2.1580,2.1580,2.158,2.158,2.158,2.158,2.158000,2.158000,2.1580,2.1580,2.158,2.158,2.158,2.158,2.158000,2.158000,2.1580,2.1580,2.158,2.158,2.158,2.158,2.158000,2.158,2.158,2.158
270,0.278,84.887,6.096,0.407,24.39,3,182.646,649.114,4.5,208.75,213.25,81.818,84.887,88.991,84.887,85.4832,84.8870,84.8870,84.8870,81.802,84.887,92.352,84.887,85.938571,84.887000,84.8870,84.8870,84.887,84.887,84.887,84.887,84.887000,84.887,84.8870,84.887,84.887,84.887,84.887,84.887,84.887000,84.887,84.887,84.887,5.513,6.096,6.501,6.096,6.0306,6.0960,6.0960,6.0960,5.315,6.096,6.684,6.096,6.021714,6.096000,6.0960,6.0960,6.096,6.096,6.096,6.096,6.096000,6.096000,6.0960,6.0960,6.096,6.096,6.096,6.096,6.096000,6.096,6.096,6.096
271,0.306,88.310,5.685,0.433,24.39,3,180.146,649.114,7.0,206.25,213.25,84.023,84.023,90.833,90.932,87.8112,87.2602,87.2602,87.8112,84.023,88.310,93.280,88.310,88.336143,88.310000,88.3100,88.3100,84.023,88.310,122.088,88.310,94.157000,88.310,88.3100,88.310,88.310,88.310,88.310,88.310,88.310000,88.310,88.310,88.310,4.574,5.685,6.229,6.229,5.5476,5.9902,5.9902,5.5476,3.852,5.685,6.229,5.685,5.387429,5.685000,5.6850,5.6850,3.360,5.685,6.229,5.685,5.055545,5.685000,5.6850,5.6850,5.685,5.685,5.685,5.685,5.685000,5.685,5.685,5.685
272,0.299,73.905,6.069,0.429,24.39,3,177.646,649.114,9.5,203.75,213.25,73.905,73.905,86.838,125.449,79.9416,95.2106,95.2106,79.9416,73.905,73.905,111.954,125.449,85.727143,100.845286,110.8210,89.9052,73.905,73.905,125.449,73.905,91.982727,73.905,73.9050,73.905,73.905,73.905,73.905,73.905,73.905000,73.90

In [111]:
print(len(test_y))
test_y.head()

61296


269    0
270    0
271    0
272    0
273    0
Name: cat_isTopMcMrNearby_known, dtype: int64

-------------------

# Machine-learning

In [112]:

seed = 123

In [113]:
# .values.ravel()
model = XGBClassifier(
    gamma=0, 
    reg_alpha=0.2, 
    max_depth=3, 
    subsample=0.8, 
    colsample_bytree= 0.8, 
    n_estimators= 300, 
    learning_rate= 0.03, 
    min_child_weight= 3,n_jobs=8)
model.fit(train_X,train_y)
result = model.predict(test_X)
result

array([ 0,  0,  0, ..., 60, 60, 60])

In [114]:
type(result)

numpy.ndarray

In [115]:
len(result)

61296

In [117]:
# df_r = pd.DataFrame(result)
# df_a = pd.DataFrame(a_)
test_y_indexValues = test_y.index.values
df_result = pd.DataFrame(result, index=test_y_indexValues, columns=['TopMcMr_Pick_pred'])
df_results_2 = pd.concat([test_y, df_result], axis=1)

In [118]:
df_results_2.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 61296 entries, 269 to 307629
Data columns (total 2 columns):
cat_isTopMcMrNearby_known    61296 non-null int64
TopMcMr_Pick_pred            61296 non-null int64
dtypes: int64(2)
memory usage: 1.4 MB


In [119]:
df_results_2.head()

,cat_isTopMcMrNearby_known,TopMcMr_Pick_pred
269,0,0
270,0,0
271,0,0
272,0,0
273,0,0


In [ ]:
# test_df_pred = test_y.copy()
# test_df_pred['Pick_pred'] = result
# test_df_pred.head()

In [120]:
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score

# make predictions for test data
# y_pred = model.predict(X_test)
# predictions = [round(value) for value in y_pred]
accuracy = accuracy_score(df_results_2['cat_isTopMcMrNearby_known'], df_results_2['TopMcMr_Pick_pred'])

In [121]:
accuracy

0.66040198381623594

--------------

In [122]:
df_results_2_onlyClass95and100 = df_results_2[df_results_2['cat_isTopMcMrNearby_known'] > 60 ]

In [124]:
df_results_2_onlyClass95and100[0:500]

,cat_isTopMcMrNearby_known,TopMcMr_Pick_pred
362,95,60
363,95,60
364,95,60
365,95,60
480,95,95
481,95,95
482,95,95
483,95,95
1165,95,100
1166,100,95


In [129]:
# df_results_2_onlyClass95and100['pred_class_95or100'] = df_results_2_onlyClass95and100[df_results_2_onlyClass95and100['cat_isTopMcMrNearby_known']]

In [126]:
df_results_2_onlyClass95and100['cat_isTopMcMrNearby_known_95or100'] = np.where(df_results_2_onlyClass95and100['cat_isTopMcMrNearby_known']>60, 1, 0)
df_results_2_onlyClass95and100['TopMcMr_Pick_pred_95or100'] = np.where(df_results_2_onlyClass95and100['TopMcMr_Pick_pred']>60, 1, 0)

/Users/justingosses/anaconda/envs/MannvilleDask/lib/python3.6/site-packages/ipykernel/__main__.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  if __name__ == '__main__':
/Users/justingosses/anaconda/envs/MannvilleDask/lib/python3.6/site-packages/ipykernel/__main__.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  from ipykernel import kernelapp as app


In [127]:
df_results_2_onlyClass95and100[0:300]

,cat_isTopMcMrNearby_known,TopMcMr_Pick_pred,cat_isTopMcMrNearby_known_95or100,TopMcMr_Pick_pred_95or100
362,95,60,1,0
363,95,60,1,0
364,95,60,1,0
365,95,60,1,0
480,95,95,1,1
481,95,95,1,1
482,95,95,1,1
483,95,95,1,1
1165,95,100,1,1
1166,100,95,1,1


In [128]:
accuracy = accuracy_score(df_results_2_onlyClass95and100['cat_isTopMcMrNearby_known_95or100'], df_results_2_onlyClass95and100['TopMcMr_Pick_pred_95or100'])
accuracy

0.57949617502314477

In [141]:
df_results_2_onlyClass95and100['cat_isTopMcMrNearby_known_60or95or100'] = np.where(df_results_2_onlyClass95and100['cat_isTopMcMrNearby_known']>59, 1, 0)
df_results_2_onlyClass95and100['TopMcMr_Pick_pred_60or95or100'] = np.where(df_results_2_onlyClass95and100['TopMcMr_Pick_pred']>59, 1, 0)
df_results_2_onlyClass95and100['cat_isTopMcMrNearby_known_100'] = np.where(df_results_2_onlyClass95and100['cat_isTopMcMrNearby_known']==100, 1, 0)
df_results_2_onlyClass95and100['TopMcMr_Pick_pred_known_100'] = np.where(df_results_2_onlyClass95and100['TopMcMr_Pick_pred']==100, 1, 0)

/Users/justingosses/anaconda/envs/MannvilleDask/lib/python3.6/site-packages/ipykernel/__main__.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  if __name__ == '__main__':
/Users/justingosses/anaconda/envs/MannvilleDask/lib/python3.6/site-packages/ipykernel/__main__.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  from ipykernel import kernelapp as app
/Users/justingosses/anaconda/envs/MannvilleDask/lib/python3.6/site-packages/ipykernel/__main__.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a Data

In [142]:
accuracy = accuracy_score(df_results_2_onlyClass95and100['cat_isTopMcMrNearby_known_60or95or100'], df_results_2_onlyClass95and100['TopMcMr_Pick_pred_60or95or100'])
accuracy

0.90839545875359351

In [143]:
accuracy = accuracy_score(df_results_2_onlyClass95and100['TopMcMr_Pick_pred_100'], df_results_2_onlyClass95and100['TopMcMr_Pick_pred_60or95or100'])
accuracy

0.39287628514349754

In [144]:
df_results_2_onlyClass95and100

,cat_isTopMcMrNearby_known,TopMcMr_Pick_pred,cat_isTopMcMrNearby_known_95or100,TopMcMr_Pick_pred_95or100,cat_isTopMcMrNearby_known_60or95or100,TopMcMr_Pick_pred_60or95or100,TopMcMr_Pick_pred_100,cat_isTopMcMrNearby_known_100,TopMcMr_Pick_pred_known_100
362,95,60,1,0,1,1,0,0,0
363,95,60,1,0,1,1,0,0,0
364,95,60,1,0,1,1,0,0,0
365,95,60,1,0,1,1,0,0,0
480,95,95,1,1,1,1,0,0,0
481,95,95,1,1,1,1,0,0,0
482,95,95,1,1,1,1,0,0,0
483,95,95,1,1,1,1,0,0,0
1165,95,100,1,1,1,1,1,0,1
1166,100,95,1,1,1,1,0,1,0


## How many times do we predict the closest row to the pick is not a "not be pick" or zero

In [83]:
df_featPlus_wUWI_testCopy = df_train_featWithHighCount[df_train_featWithHighCount['trainOrTest'] == 'test' ].copy()

In [84]:
df_featPlus_wUWI_testCopy.head()

,UWI,trainOrTest,CALI,DPHI,GR,ILD,NPHI,RHOB,SP,McMurray_Base_Qual,McMurray_Top_Qual,lat,lng,NN1_thickness,MM_Top_Depth_predBy_NN1thick,Quality,Quality_paleoz,DistFrom_NN1_TopDepth_Abs,BotWellDept,FromTopWell,FromBotWell,WellThickness,rowsToEdge,GR_min_5winSize_dirAroundMin,GR_min_5winSize_dirAboveMin,GR_min_5winSize_dirAroundMax,GR_min_5winSize_dirAboveMax,GR_min_5winSize_dirAroundMean,GR_min_5winSize_dirAboveMean,GR_min_5winSize_dirAbovenLarge,GR_min_5winSize_dirAroundnLarge,GR_min_7winSize_dirAroundMin,GR_min_7winSize_dirAboveMin,GR_min_7winSize_dirAroundMax,GR_min_7winSize_dirAboveMax,GR_min_7winSize_dirAroundMean,GR_min_7winSize_dirAboveMean,GR_min_7winSize_dirAbovenLarge,GR_min_7winSize_dirAroundnLarge,GR_min_11winSize_dirAroundMin,GR_min_11winSize_dirAboveMin,GR_min_11winSize_dirAroundMax,GR_min_11winSize_dirAboveMax,GR_min_11winSize_dirAroundMean,GR_min_11winSize_dirAboveMean,GR_min_11winSize_dirAbovenLarge,GR_min_11winSize_dirAroundnLarge,GR_min_21winSize_dirAroundMin,GR_min_21winSize_dirAboveMin,GR_min_21winSize_dirAroundMax,GR_min_21winSize_dirAboveMax,GR_min_21winSize_dirAroundMean,GR_min_21winSize_dirAboveMean,GR_min_21winSize_dirAbovenLarge,GR_min_21winSize_dirAroundnLarge,ILD_min_5winSize_dirAroundMin,ILD_min_5winSize_dirAboveMin,ILD_min_5winSize_dirAroundMax,ILD_min_5winSize_dirAboveMax,ILD_min_5winSize_dirAroundMean,ILD_min_5winSize_dirAboveMean,ILD_min_5winSize_dirAbovenLarge,ILD_min_5winSize_dirAroundnLarge,ILD_min_7winSize_dirAroundMin,ILD_min_7winSize_dirAboveMin,ILD_min_7winSize_dirAroundMax,ILD_min_7winSize_dirAboveMax,ILD_min_7winSize_dirAroundMean,ILD_min_7winSize_dirAboveMean,ILD_min_7winSize_dirAbovenLarge,ILD_min_7winSize_dirAroundnLarge,ILD_min_11winSize_dirAroundMin,ILD_min_11winSize_dirAboveMin,ILD_min_11winSize_dirAroundMax,ILD_min_11winSize_dirAboveMax,ILD_min_11winSize_dirAroundMean,ILD_min_11winSize_dirAboveMean,ILD_min_11winSize_dirAbovenLarge,ILD_min_11winSize_dirAroundnLarge,ILD_min_21winSize_dirAroundMin,ILD_min_21winSize_dirAboveMin,ILD_min_21winSize_dirAroundMax,ILD_min_21winSize_dirAboveMax,ILD_min_21winSize_dirAroundMean,ILD_min_21winSize_dirAboveMean,ILD_min_21winSize_dirAbovenLarge,ILD_min_21winSize_dirAroundnLarge
990,00/04-15-081-09W4/0,test,NaN,0.258,123.954,3.665,0.579,NaN,NaN,1,3,56.015445,-111.333199,46.0,431.32,3,1,186.73,487.34,0.75,242.75,243.5,3,123.954,123.954,123.954,123.954,123.9540,123.9540,123.9540,123.9540,123.954,123.954,123.954,123.954,123.954000,123.954000,123.9540,123.9540,123.954,123.954,123.954,123.954,123.954000,123.954,123.954,123.954,123.954,123.954,123.954,123.954,123.95400,123.954,123.954,123.954,3.665,3.665,3.665,3.665,3.6650,3.6650,3.6650,3.6650,3.665,3.665,3.665,3.665,3.665000,3.665000,3.6650,3.6650,3.665,3.665,3.665,3.665,3.665000,3.665,3.665,3.665,3.665,3.665,3.665,3.665,3.66500,3.665,3.665,3.665
1000,00/04-15-081-09W4/0,test,NaN,0.280,123.234,3.619,0.596,NaN,NaN,1,3,56.015445,-111.333199,46.0,431.32,3,1,184.23,487.34,3.25,240.25,243.5,13,115.836,123.234,126.644,123.234,121.4082,123.2340,123.2340,123.2340,115.836,123.234,126.644,123.234,121.248286,123.234000,123.2340,123.2340,123.234,123.234,123.234,123.234,123.234000,123.234,123.234,123.234,123.234,123.234,123.234,123.234,123.23400,123.234,123.234,123.234,3.584,3.619,3.666,3.619,3.6280,3.6190,3.6190,3.6190,3.584,3.619,3.721,3.619,3.648571,3.619000,3.6190,3.6190,3.619,3.619,3.619,3.619,3.619000,3.619,3.619,3.619,3.619,3.619,3.619,3.619,3.61900,3.619,3.619,3.619
1010,00/04-15-081-09W4/0,test,NaN,0.273,112.040,3.951,0.561,NaN,NaN,1,3,56.015445,-111.333199,46.0,431.32,3,1,181.73,487.34,5.75,237.75,243.5,23,110.998,110.998,112.426,115.666,111.8636,113.0700,113.0700,111.8636,110.630,112.040,114.443,112.040,112.055857,112.040000,112.0400,112.0400,110.492,112.040,115.969,112.040,112.553727,112.040,112.040,112.040,112.040,112.040,112.040,112.040,112.04000,112.040,112.040,112.040,3.890,3.868,3.951,3.951,3.9372,3.9082,3.9082,3.9372,3.868,3.951,3.957,3.951,3.930143,3.951000,3.9510,3.9510,3.807,3.951,3.957,3.951,3

In [90]:
df_featPlus_wUWI_testCopy_wResults = pd.concat([df_featPlus_wUWI_testCopy, df_results_2], axis=1)
df_featPlus_wUWI_testCopy_wResults.tail()

,UWI,trainOrTest,CALI,DPHI,GR,ILD,NPHI,RHOB,SP,McMurray_Base_Qual,McMurray_Top_Qual,lat,lng,NN1_thickness,MM_Top_Depth_predBy_NN1thick,Quality,Quality_paleoz,DistFrom_NN1_TopDepth_Abs,BotWellDept,FromTopWell,FromBotWell,WellThickness,rowsToEdge,GR_min_5winSize_dirAroundMin,GR_min_5winSize_dirAboveMin,GR_min_5winSize_dirAroundMax,GR_min_5winSize_dirAboveMax,GR_min_5winSize_dirAroundMean,GR_min_5winSize_dirAboveMean,GR_min_5winSize_dirAbovenLarge,GR_min_5winSize_dirAroundnLarge,GR_min_7winSize_dirAroundMin,GR_min_7winSize_dirAboveMin,GR_min_7winSize_dirAroundMax,GR_min_7winSize_dirAboveMax,GR_min_7winSize_dirAroundMean,GR_min_7winSize_dirAboveMean,GR_min_7winSize_dirAbovenLarge,GR_min_7winSize_dirAroundnLarge,GR_min_11winSize_dirAroundMin,GR_min_11winSize_dirAboveMin,GR_min_11winSize_dirAroundMax,GR_min_11winSize_dirAboveMax,GR_min_11winSize_dirAroundMean,GR_min_11winSize_dirAboveMean,GR_min_11winSize_dirAbovenLarge,GR_min_11winSize_dirAroundnLarge,GR_min_21winSize_dirAroundMin,GR_min_21winSize_dirAboveMin,GR_min_21winSize_dirAroundMax,GR_min_21winSize_dirAboveMax,GR_min_21winSize_dirAroundMean,GR_min_21winSize_dirAboveMean,GR_min_21winSize_dirAbovenLarge,GR_min_21winSize_dirAroundnLarge,ILD_min_5winSize_dirAroundMin,ILD_min_5winSize_dirAboveMin,ILD_min_5winSize_dirAroundMax,ILD_min_5winSize_dirAboveMax,ILD_min_5winSize_dirAroundMean,ILD_min_5winSize_dirAboveMean,ILD_min_5winSize_dirAbovenLarge,ILD_min_5winSize_dirAroundnLarge,ILD_min_7winSize_dirAroundMin,ILD_min_7winSize_dirAboveMin,ILD_min_7winSize_dirAroundMax,ILD_min_7winSize_dirAboveMax,ILD_min_7winSize_dirAroundMean,ILD_min_7winSize_dirAboveMean,ILD_min_7winSize_dirAbovenLarge,ILD_min_7winSize_dirAroundnLarge,ILD_min_11winSize_dirAroundMin,ILD_min_11winSize_dirAboveMin,ILD_min_11winSize_dirAroundMax,ILD_min_11winSize_dirAboveMax,ILD_min_11winSize_dirAroundMean,ILD_min_11winSize_dirAboveMean,ILD_min_11winSize_dirAbovenLarge,ILD_min_11winSize_dirAroundnLarge,ILD_min_21winSize_dirAroundMin,ILD_min_21winSize_dirAboveMin,ILD_min_21winSize_dirAroundMax,ILD_min_21winSize_dirAboveMax,ILD_min_21winSize_dirAroundMean,ILD_min_21winSize_dirAboveMean,ILD_min_21winSize_dirAbovenLarge,ILD_min_21winSize_dirAroundnLarge,cat_isTopMcMrNearby_known,TopMcMr_Pick_pred
1473050,00/06-22-078-04W4/0,test,NaN,0.308,130.061,3.685,0.621,2183.47,NaN,1,1,55.771708,-110.539294,86.56,306.44,1,1,81.56,400.0,248.0,12.0,260.0,48,120.035,105.744,136.340,136.340,128.6864,127.4644,127.4644,128.6864,120.035,79.523,136.340,136.340,128.454143,113.854000,127.4644,131.6942,80.133,28.332,136.340,136.340,122.504727,92.121909,127.4644,133.044,28.332,130.061,136.340,130.061,99.973762,130.061,130.061,130.061,3.481,3.554,4.130,4.130,3.7786,3.8648,3.8648,3.7786,3.481,2.837,4.130,4.130,3.782429,3.619429,3.8648,3.890,3.175,1.911,4.276,4.130,3.754727,3.084545,3.8648,4.0362,1.911,3.685,4.783,3.685,3.600143,3.685,3.685,3.685,0,0
1473060,00/06-22-078-04W4/0,test,NaN,0.240,34.303,4.348,0.340,2299.34,NaN,1,1,55.771708,-110.539294,86.56,306.44,1,1,84.06,400.0,250.5,9.5,260.0,38,19.898,34.303,93.702,133.728,49.3010,91.2022,91.2022,49.3010,16.760,34.303,119.964,133.728,54.747000,102.643857,121.9780,69.3142,16.743,34.303,133.728,34.303,62.083818,34.303000,34.3030,34.303,34.303,34.303,34.303,34.303,34.303000,34.303,34.303,34.303,3.036,4.348,4.783,4.783,4.1160,4.5740,4.5740,4.1160,2.466,3.820,4.783,4.783,3.963143,4.423714,4.5740,4.448,2.079,4.348,4.783,4.348,3.707182,4.348000,4.3480,4.3480,4.348,4.348,4.348,4.348,4.348000,4.348,4.348,4.348,0,0
1473070,00/06-22-078-04W4/0,test,NaN,0.593,128.531,4.583,0.571,1695.98,NaN,1,1,55.771708,-110.539294,86.56,306.44,1,1,86.56,400.0,253.0,7.0,260.0,28,41.690,22.341,128.531,128.531,92.2838,57.3134,57.3134,92.2838,29.205,128.531,128.531,128.531,83.088000,128.531000,128.5310,128.5310,17.980,128.531,128.531,128.531,71.829909,128.531000,128.5310,128.531,128.531,128.531,128.531,128.531,128.531000,128.531,128.531,128.531,3.144,2.039,6.192,4.583,4.6494,3.2140,3.2140,4.6494,2.508,4.583,6.192,4.583,4.547000

In [126]:
wells_in_test = df_featPlus_wUWI_testCopy_wResults['UWI'].unique()
len(wells_in_test)

382

In [81]:
import pdvega
import vega

In [136]:
df_look_at_pred_class_vs_distFromRealLess1 = df_featPlus_wUWI_testCopy_wResults[df_featPlus_wUWI_testCopy_wResults['DistFrom_NN1_TopDepth_Abs'] < 1 ]

In [137]:
df_look_at_pred_class_vs_distFromRealLess1['cat_isTopMcMrNearby_known'].nunique()

4

In [138]:
df_count = df_look_at_pred_class_vs_distFromRealLess1.groupby('cat_isTopMcMrNearby_known').nunique()

In [139]:
df_count

,UWI,trainOrTest,CALI,DPHI,GR,ILD,NPHI,RHOB,SP,McMurray_Base_Qual,McMurray_Top_Qual,lat,lng,NN1_thickness,MM_Top_Depth_predBy_NN1thick,Quality,Quality_paleoz,DistFrom_NN1_TopDepth_Abs,BotWellDept,FromTopWell,FromBotWell,WellThickness,rowsToEdge,GR_min_5winSize_dirAroundMin,GR_min_5winSize_dirAboveMin,GR_min_5winSize_dirAroundMax,GR_min_5winSize_dirAboveMax,GR_min_5winSize_dirAroundMean,GR_min_5winSize_dirAboveMean,GR_min_5winSize_dirAbovenLarge,GR_min_5winSize_dirAroundnLarge,GR_min_7winSize_dirAroundMin,GR_min_7winSize_dirAboveMin,GR_min_7winSize_dirAroundMax,GR_min_7winSize_dirAboveMax,GR_min_7winSize_dirAroundMean,GR_min_7winSize_dirAboveMean,GR_min_7winSize_dirAbovenLarge,GR_min_7winSize_dirAroundnLarge,GR_min_11winSize_dirAroundMin,GR_min_11winSize_dirAboveMin,GR_min_11winSize_dirAroundMax,GR_min_11winSize_dirAboveMax,GR_min_11winSize_dirAroundMean,GR_min_11winSize_dirAboveMean,GR_min_11winSize_dirAbovenLarge,GR_min_11winSize_dirAroundnLarge,GR_min_21winSize_dirAroundMin,GR_min_21winSize_dirAboveMin,GR_min_21winSize_dirAroundMax,GR_min_21winSize_dirAboveMax,GR_min_21winSize_dirAroundMean,GR_min_21winSize_dirAboveMean,GR_min_21winSize_dirAbovenLarge,GR_min_21winSize_dirAroundnLarge,ILD_min_5winSize_dirAroundMin,ILD_min_5winSize_dirAboveMin,ILD_min_5winSize_dirAroundMax,ILD_min_5winSize_dirAboveMax,ILD_min_5winSize_dirAroundMean,ILD_min_5winSize_dirAboveMean,ILD_min_5winSize_dirAbovenLarge,ILD_min_5winSize_dirAroundnLarge,ILD_min_7winSize_dirAroundMin,ILD_min_7winSize_dirAboveMin,ILD_min_7winSize_dirAroundMax,ILD_min_7winSize_dirAboveMax,ILD_min_7winSize_dirAroundMean,ILD_min_7winSize_dirAboveMean,ILD_min_7winSize_dirAbovenLarge,ILD_min_7winSize_dirAroundnLarge,ILD_min_11winSize_dirAroundMin,ILD_min_11winSize_dirAboveMin,ILD_min_11winSize_dirAroundMax,ILD_min_11winSize_dirAboveMax,ILD_min_11winSize_dirAroundMean,ILD_min_11winSize_dirAboveMean,ILD_min_11winSize_dirAbovenLarge,ILD_min_11winSize_dirAroundnLarge,ILD_min_21winSize_dirAroundMin,ILD_min_21winSize_dirAboveMin,ILD_min_21winSize_dirAroundMax,ILD_min_21winSize_dirAboveMax,ILD_min_21winSize_dirAroundMean,ILD_min_21winSize_dirAboveMean,ILD_min_21winSize_dirAbovenLarge,ILD_min_21winSize_dirAroundnLarge,cat_isTopMcMrNearby_known,TopMcMr_Pick_pred
cat_isTopMcMrNearby_known,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
0,158,1,50,101,157,155,112,13,1,4,3,157,157,128,153,3,4,87,134,146,137,104,128,157,157,156,157,157,157,157,157,157,156,156,157,157,157,157,157,156,156,157,157,157,157,157,157,156,156,157,157,157,157,157,157,156,155,154,155,156,156,156,156,156,155,156,155,156,156,156,156,156,156,156,156,156,156,156,156,156,156,156,156,156,156,156,156,1,2
60,149,1,275,281,858,825,285,85,8,4,3,149,148,113,146,3,4,381,134,547,435,106,304,586,592,586,588,865,865,865,865,511,535,515,527,865,865,806,794,441,483,447,509,864,864,727,713,384,466,430,485,865,865,645,645,654,671,639,650,850,847,846,846,600,616,566,593,850,846,782,779,530,567,474,530,847,847,705,673,483,526,393,528,846,847,645,565,1,2
95,43,1,34,66,85,84,66,9,0,4,3,43,42,39,42,3,4,53,38,79,79,35,76,77,77,62,73,85,85,85,85,74,71,61,72,85,85,81,81,72,57,64,74,85,85,81,76,53,64,58,74,85,85,79,76,67,74,83,78,84,84,84,84,63,75,83,68,85,84,80,85,60,77,72,63,85,84,70,83,64,76,62,67,84,84,70,69,1,1
100,17,1,8,17,17,17,16,1,0,2,3,17,17,16,17,3,2,6,15,17,17,16,16,17,17,17,17,17,17,17,17,17,17,17,17,17,17,17,17,17,17,17,17,17,17,17,17,17,17,17,17,17,17,17,17,17,17,17,17,17,17,17,17,17,17,17,17,17,17,17,17,17,17,17,17,17,17,17,17,17,17,17,17,17,17,17,17,1,1


In [140]:
total_rows_less_than_1_from_pick = df_count['UWI'].unique().sum()
total_rows_less_than_1_from_pick

367

In [141]:
df_count['UWI']

cat_isTopMcMrNearby_known
0      158
60     149
95      43
100     17
Name: UWI, dtype: int64

In [142]:
def getPercents(df,total_wells):
    for Each in df:
        print(Each, "total or the % is: ",Each/total_wells)
    
    

In [143]:
getPercents(df_count['UWI'],total_rows_less_than_1_from_pick)

158 total or the % is:  0.430517711172
149 total or the % is:  0.405994550409
43 total or the % is:  0.117166212534
17 total or the % is:  0.0463215258856


In [ ]:
# 'MM_Top_Depth_predBy_NN1thick'
test_df_pred2_TopScratch2 = test_df_pred_onlyTopMCM_class5[['DistFrom_NN1_TopDepth']]
test_df_pred2_TopScratch2.vgplot.hist(bins=100, alpha=0.5)

In [ ]:
# client = Client()
# client

In [ ]:
# client.close()

In [ ]:
# from xgboost.sklearn import XGBClassifier
# from dask_ml.xgboost import XGBRegressor


# from dask_ml.xgboost import XGBClassifier

In [ ]:
# train_X_dd = dd.from_pandas(train_X, npartitions=10)
# train_y_dd = dd.from_pandas(train_y, npartitions=10)

In [ ]:
# est = XGBClassifier(gamma=0, 
#     reg_alpha=0.2, 
#     max_depth=3, 
#     subsample=0.8, 
#     colsample_bytree= 0.8, 
#     n_estimators= 300, 
#     learning_rate= 0.03, 
#     min_child_weight= 3)
# est.fit(train_X,train_y)

## What types of normalization are appropriate? 